In [10]:
import sys
import os.path as path
import tensorflow as tf
import torch
from IPython.display import clear_output

sys.path.append(path.join('drive','My Drive','Colab Notebooks'))

device_name = tf.test.gpu_device_name()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

clear_output()
if device_name == '/device:GPU:0':
  print('GPU: ',torch.cuda.get_device_name(0))

GPU:  Tesla T4


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
%cd '/content/drive/My Drive/Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [13]:
import os
import random
import timeit
import json
import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm.notebook import tqdm
!pip install transformers
!pip install -q sentencepiece
from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    squad_convert_examples_to_features,
)
from transformers.data.metrics.squad_metrics import (
    compute_predictions_logits,
    squad_evaluate,
)
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

"""Parameters"""


model_name_or_path = 'm3hrdadfi/albert-fa-base-v2'
#model_name_or_path = 'HooshvareLab/bert-base-parsbert-uncased'
output_dir = '/content/drive/MyDrive/HW_5/Dataset'
dataset_name = ''
version_2 = False
null_score_diff_threshold = 0
max_seq_length = 512
max_query_length = 64
doc_stride = 128
do_lowercase = False
per_gpu_train_batch_size = 12
per_gpu_eval_batch_size = 8
learning_rate = 3e-5
gradient_accumulation_steps = 1
weight_decay = 0
max_grad_norm = 1
num_train_epochs = 1
warmup_steps = 0
n_best_size = 20
max_answer_length = 30
seed = 42
threads = 10

"""Parameters"""


def set_seed():
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)


def to_list(tensor):
    return tensor.detach().cpu().tolist()


def train(train_dataset, model, tokenizer, start_epoch):
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=per_gpu_train_batch_size)

    t_total = len(train_dataloader) // gradient_accumulation_steps * num_train_epochs

    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
    )

    if os.path.isfile(os.path.join(output_dir, "optimizer.pt")) and os.path.isfile(
        os.path.join(output_dir, "scheduler.pt")
    ):
        optimizer.load_state_dict(torch.load(os.path.join(output_dir, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(output_dir, "scheduler.pt")))

    print("\nTraining:")

    global_step = 1
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    set_seed()

    for epoch_idx in range(num_train_epochs):
        if epoch_idx < start_epoch:
            continue
        epoch_iterator = tqdm(train_dataloader, desc="Iteration in epoch {}".format(epoch_idx+1))
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(device) for t in batch)
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
                "start_positions": batch[3],
                "end_positions": batch[4],
            }
            outputs = model(**inputs)
            loss = outputs[0]
            if n_gpu > 1:
                loss = loss.mean()
            if gradient_accumulation_steps > 1:
                loss = loss / gradient_accumulation_steps
            loss.backward()
            tr_loss += loss.item()
            if (step + 1) % gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                optimizer.step()
                scheduler.step()
                model.zero_grad()
                global_step += 1
        checkpoint_output_dir = os.path.join(output_dir, 'checkpoint_{}'.format(epoch_idx+1))
        model_to_save = model.module if hasattr(model, "module") else model
        model_to_save.save_pretrained(checkpoint_output_dir)
        tokenizer.save_pretrained(checkpoint_output_dir)
        torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
        torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
    return global_step, tr_loss / global_step


def evaluate(model, tokenizer):
    dataset, examples, features = load_and_cache_examples(tokenizer, evaluate=True, output_examples=True)

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=per_gpu_eval_batch_size)

    print("\nEvaluation:")

    all_results = []
    start_time = timeit.default_timer()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }
            feature_indices = batch[3]
            outputs = model(**inputs, return_dict=False)

            for i, feature_index in enumerate(feature_indices):
                eval_feature = features[feature_index.item()]
                unique_id = int(eval_feature.unique_id)
                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)

                all_results.append(result)

    evalTime = timeit.default_timer() - start_time
    print("  Evaluation done in total {} secs ({} sec per example)".format(evalTime, evalTime / len(dataset)))

    output_prediction_file = os.path.join(output_dir, "predictions.json")
    output_nbest_file = os.path.join(output_dir, "nbest_predictions.json")

    if version_2:
        output_null_log_odds_file = os.path.join(output_dir, "null_odds.json")
    else:
        output_null_log_odds_file = None
    
    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lowercase,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,
        version_2,
        null_score_diff_threshold,
        tokenizer,
    )

    if output_null_log_odds_file is not None:
        filename = os.path.join(output_dir, 'null_odds.json')
        null_odds = json.load(open(filename, 'rb'))
    else:
        null_odds = None
    results = squad_evaluate(examples, predictions, no_answer_probs=null_odds, no_answer_probability_threshold=null_score_diff_threshold)
    return results


def load_and_cache_examples(tokenizer, evaluate=False, output_examples=False):
    input_dir = os.path.join('/content/drive/MyDrive/HW_5/Dataset/', dataset_name)
    cached_features_file = os.path.join(
        input_dir,
        "cached_{}".format("test" if evaluate else "train"),
    )

    if os.path.exists(cached_features_file):
        print("Loading features from cached file {}".format(cached_features_file))
        features_and_dataset = torch.load(cached_features_file)
        features, dataset, examples = (
            features_and_dataset["features"],
            features_and_dataset["dataset"],
            features_and_dataset["examples"],
        )
    else:
        print("Creating features from dataset file at {}".format(input_dir))
        processor = SquadV2Processor() if version_2 else SquadV1Processor()
        if evaluate:
            examples = processor.get_dev_examples(input_dir, filename='Test.json')
        else:
            examples = processor.get_train_examples(input_dir, filename='Train.json')
        features, dataset = squad_convert_examples_to_features(
            examples=examples,
            tokenizer=tokenizer,
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_training=not evaluate,
            return_dataset="pt",
            threads=threads,
        )
        print("Saving features into cached file {}".format(cached_features_file))
        torch.save({"features": features, "dataset": dataset, "examples": examples}, cached_features_file)

    if output_examples:
        return dataset, examples, features
    return dataset


def main():
    set_seed()

    global_step = ""

    model_file = os.path.join(output_dir,'pytorch_model.bin')
    do_train = not os.path.exists(model_file)
    if do_train:
        checkpoints_dir = filter(lambda x:x.startswith('checkpoint_'), os.listdir(output_dir))
        checkpoint = max(map(lambda x:int(x[x.find('_')+1:]), checkpoints_dir), default=0)
        if checkpoint == 0:
            config = AutoConfig.from_pretrained(model_name_or_path)
            tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, do_lower_case=do_lowercase, use_fast=False)
            train_dataset = load_and_cache_examples(tokenizer, evaluate=False, output_examples=False)
            model = AutoModel.from_pretrained(model_name_or_path, config=config)
        else:
            checkpoint_output_dir = os.path.join(output_dir, 'checkpoint_{}'.format(checkpoint))
            config = AutoConfig.from_pretrained(checkpoint_output_dir)
            tokenizer = AutoTokenizer.from_pretrained(checkpoint_output_dir, do_lower_case=do_lowercase, use_fast=False)
            train_dataset = load_and_cache_examples(tokenizer, evaluate=False, output_examples=False)
            model = AutoModel.from_pretrained(checkpoint_output_dir, config=config)
        model.to(device)
        global_step, tr_loss = train(train_dataset, model, tokenizer, checkpoint)
        print(" global_step = {}, average loss = {}".format(global_step, tr_loss))
        print("Saving model checkpoint to {}".format(output_dir))
        model_to_save = model.module if hasattr(model, "module") else model
        model_to_save.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)

    result = {}
    print("Evaluate the last checkpoint")
    config = AutoConfig.from_pretrained(output_dir)
    tokenizer = AutoTokenizer.from_pretrained(output_dir, do_lower_case=do_lowercase, use_fast=False)
    model = AutoModel.from_pretrained(output_dir, config=config)
    model.to(device)
    eval_result = evaluate(model, tokenizer)
    result = dict((k, v) for k, v in eval_result.items())

    print(result)
    print("\nResult:\n\texact_match: {}\n\tf1: {}".format(result['exact'], result['f1']))
    return result


if __name__ == "__main__":
    main()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Evaluate the last checkpoint
Loading features from cached file /content/drive/MyDrive/HW_5/Dataset/cached_test

Evaluation:


Evaluating:   0%|          | 0/1021 [00:00<?, ?it/s]

  Evaluation done in total 213.4106203199999 secs (0.026137246824249834 sec per example)
{'exact': 50.44988752811797, 'f1': 64.65321757161851, 'total': 8002, 'HasAns_exact': 66.22864651773982, 'HasAns_f1': 84.89734674902944, 'HasAns_total': 6088, 'NoAns_exact': 0.2612330198537095, 'NoAns_f1': 0.2612330198537095, 'NoAns_total': 1914, 'best_exact': 50.39990002499375, 'best_exact_thresh': 0.0, 'best_f1': 64.60323006849427, 'best_f1_thresh': 0.0}

Result:
	exact_match: 50.44988752811797
	f1: 64.65321757161851
